<a href="https://colab.research.google.com/github/daniel-trezena/Ingestao_api_spotify/blob/main/Teste_Spark_Autoglass_Daniel_Trezena.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Teste: Consumo de Dados da Api do Spotify para analises sobre artistas e músicas.

## Objetivo

Avaliar conhecimentos nas linguagens Python e SQL e na engine de processamento Apache Spark.

## Descrição

Dado um conjunto de dados de músicas e artistas disponíveis em uma API pública, implemente um pipeline de processamento de dados com PySpark que realiza as seguintes operações:

* Consuma a API e extraia as informações de músicas e artistas dos seguintes genêros("Rock Nacional", "Piseiro/Arrocha" e "Pop Internacional";

* Armazene os dados em formato parquet, particionando por artista;

* Crie um dataframe com o endpoint "Get Artist"
* Crie um dataframe com o endpoint "Get Artist's Albums" e traga as músicas dos álbuns que estão no endpoint "Get Album Tracks".
* Crie um dataframe com o endpoint "Get Current User's Playlists"

* Crie uma tabela temporária em PySpark a partir do DataFrame de músicas dos artistas;
* Crie uma tabela temporária em Pyspark a partir do Dataframe de Playlists

* Execute uma consulta SQL que retorna os artistas do endpoint "Get Artist" que estão nas Playlists, ordenados por ordem alfabética.
* Crie um Dataframe com o resultado da consulta e salve em parquet

.

-------

Documentação da Api para consulta:
```
https://developer.spotify.com/documentation/web-api/
```

Para criar um Access Token na Spotify API, você precisa seguir os seguintes passos:

Acesse o Dashboard da API Spotify:
```
https://developer.spotify.com/dashboard/login
```
1. Faça login com sua conta do Spotify ou crie uma nova conta, caso ainda não tenha.
2. Crie um novo aplicativo clicando no botão "Create an App" e preencha as informações necessárias.
3. Após criar o aplicativo, você será direcionado para a página do aplicativo, onde poderá encontrar sua Client ID e Client Secret. Anote essas informações, pois elas serão necessárias para a autenticação.
4. Para gerar o Access Token, você precisará fazer uma solicitação GET/POST para o endpoint de autorização da API, passando sua Client ID e Client Secret como parâmetros. Você pode usar ferramentas como o Postman para realizar essa solicitação.
5. O endpoint de autorização irá retornar um Access Token que você pode utilizar para fazer requisições à API Spotify.

------------


## Atenção

- Leia a documentação dos endpoints para a extração dos dados, em alguns podem possuir número máximo de itens a serem retornados.

### Para realizar esta tarefa, os candidatos devem ter conhecimento em:

- PySpark (Spark SQL, Spark DataFrames);
- Consumo de API REST;
- armazenamento de dados;
- Transformações e consultas SQL em PySpark.
-------------
## Entregando o desafio

Faça uma cópia do desafio antes de começar a fazer o desafio e depois exporte para enviar!

Concluindo todos os passos informados, basta salvar o arquivo .ipynb do notebook e zipar juntamente com os parquet das tabelas, postar no seu github e enviar o link para ricardo.suhete@autoglass.com.br

A entrega da tarefa termina 7 dias após o recebimento deste notebook. Após o prazo será entendindo que o condidato desistiu da vaga.



-----------------------------------------------
### Caso não consiga rodar o spark no colab execute os comandos abaixo

In [1]:
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q https://dlcdn.apache.org/spark/spark-3.3.1/spark-3.3.1-bin-hadoop3.tgz
!tar xf spark-3.3.1-bin-hadoop3.tgz

tar: spark-3.3.1-bin-hadoop3.tgz: Cannot open: No such file or directory
tar: Error is not recoverable: exiting now


In [2]:
#import os
#os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
#os.environ["SPARK_HOME"] = "/content/spark-3.3.1-bin-hadoop3"

---------------------------------------------------

### Instalação das bibliotecas

In [3]:
!pip install -q findspark
!pip install -q unidecode
!pip install -q pyspark

### **Importação das bibliotecas**

In [4]:
import findspark
findspark.init()

import requests
import os
import json
import unidecode
import base64
from pyspark.sql import SparkSession


from google.colab import userdata

spark = SparkSession.builder \
      .master("local[1]") \
      .appName("Teste_Spark_Autoglass_candidatos") \
      .getOrCreate()
from pyspark.sql.types import StructType, StringType

### Requisição do Token

In [5]:
client_id = userdata.get('Client_ID')
client_secret = userdata.get('Client_secret')

In [6]:
def get_token():
  auth_string = client_id + ":" + client_secret
  auth_bytes = auth_string.encode("ascii")

  base64_bytes = base64.b64encode(auth_bytes)
  auth_base64 = base64_bytes.decode('ascii')

  url = "https://accounts.spotify.com/api/token"
  headers = {
      "Authorization" : "Basic " + auth_base64,
      "Content-Type" : "application/x-www-form-urlencoded"
  }
  data = {"grant_type": "client_credentials"}
  result = requests.request('POST', url=url, headers=headers, data=data)
  token = result.json()["access_token"]
  return token

In [7]:
token = get_token()

### Requisição de dados de músicas e artistas dos seguintes genêros("Rock Nacional", "Piseiro/Arrocha" e "Pop Internacional"

In [8]:
# Configurando headers para requisições
headers = {
    "Authorization": "Bearer " + token,
    "Accept": "application/json"
}

In [9]:
generos = ["rock nacional", "piseiro/arrocha", "pop internacional"]

for genero in generos:

  # Variáveis para paginação
  offset = 0
  limit = 50
  musicas_lista = []

  while True:
      # Parâmetros
      url = "https://api.spotify.com/v1/search"
      params = {
          "q": f"genre:{genero}",
          "type": "track",
          "limit": limit,
          "offset": offset,
      }

      # Fazer requisição
      resposta = requests.get(url, params=params, headers=headers)

      # Extração dos dados
      musicas = resposta.json()["tracks"]["items"]

      # Extrair informações
      for musica in musicas:
        musica_dict = {
          "Nome": musica["name"],
          "Artista": musica["artists"][0]["name"],
          "Álbum": musica["album"]["name"],
          "Link": musica["external_urls"]["spotify"],
      }
        musicas_lista.append(musica_dict)

      # Verificar se há mais resultados
      if resposta.json()["tracks"]["next"] is None:
          break

      # Atualizar offset
      offset += limit

df_musicas = spark.createDataFrame(musicas_lista)
df_musicas.show()

+----------------+--------------------+--------------------+--------------------+
|         Artista|                Link|                Nome|               Álbum|
+----------------+--------------------+--------------------+--------------------+
|     Ana Castela|https://open.spot...|Solteiro Forçado ...|Solteiro Forçado ...|
|     Ana Castela|https://open.spot...|Tô Voltando - (Bo...|Tô Voltando (Boia...|
|   Bomba Estéreo|https://open.spot...|     Internacionales|                 Ayo|
|     Ana Castela|https://open.spot...| Fronteira - Ao Vivo|Boiadeira Interna...|
|     Ana Castela|https://open.spot...|Aqui Tem Alguém -...|Boiadeira Interna...|
|El Gran Silencio|https://open.spot...|Super Riddim Inte...|Super Riddim Inte...|
|     Ana Castela|https://open.spot...| Fronteira - Ao Vivo|Fronteira: Boiade...|
|     Darío Gómez|https://open.spot...|         Sobreviviré|El Rey del Despec...|
|     Darío Gómez|https://open.spot...|      Entre Comillas|El Rey del Despec...|
|     Ana Castel

### Armazenamento dos dados em formato parquet, particionando por artista

In [10]:
df_musicas.write.option("header",True).partitionBy("Artista").mode("overwrite").parquet("parquet_track")

### Dataframe com os dados do endpoint "Get Artist"

In [11]:
artist = "4fdCGYM7dtJLa3LvR1ccto?si=WP9igjJ9Te2yqsAm12WkKw"
url = f"https://api.spotify.com/v1/artists/{artist}"
resposta = requests.get(url, headers=headers)

In [12]:
artista_dict = json.dumps(resposta.json())
df_artista = spark.read.json(spark.sparkContext.parallelize([artista_dict]))
df_artista.show(truncate=False)

+--------------------------------------------------------+---------------+----------------------------+---------------------------------------------------------+----------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------+----------+------+-------------------------------------+
|external_urls                                           |followers      |genres                      |href                                                     |id                    |images                                                                                                                                                                                                                                    |name          |popularity|type  |uri                                  |
+-----------------

### Dataframe com os dados do endpoint "Get Artist's Albums"

In [13]:
url = f"https://api.spotify.com/v1/artists/{artist}/albums"
resposta = requests.get(url, headers=headers)
resposta_dict = json.dumps(resposta.json())
df_artista_album = spark.read.json(spark.sparkContext.parallelize([resposta_dict]))

In [14]:
df_artista_album.show()

+--------------------+---------------+--------------------+--------------------+--------------------+--------------------+--------------+----------+------+--------------------+
|       external_urls|      followers|              genres|                href|                  id|              images|          name|popularity|  type|                 uri|
+--------------------+---------------+--------------------+--------------------+--------------------+--------------------+--------------+----------+------+--------------------+
|{https://open.spo...|{NULL, 6084601}|[adoracao, brazil...|https://api.spoti...|4fdCGYM7dtJLa3LvR...|[{640, https://i....|Gabriela Rocha|        70|artist|spotify:artist:4f...|
+--------------------+---------------+--------------------+--------------------+--------------------+--------------------+--------------+----------+------+--------------------+



### Músicas dos álbuns que estão no endpoint "Get Album Tracks"

In [15]:
album_id = "4aawyAB9vmqN3uQ7FjRGTy"
url = f"https://api.spotify.com/v1/albums/{album_id}/tracks"
params = {
          "market": "ES",
          "limit": 50,
          "offset": 0,
      }

# Fazer requisição
resposta = requests.get(url, params=params, headers=headers)

# Extração dos dados
musicas = resposta.json()["items"]
musicas_lista_album = []
  # Extrair informações
for musica in musicas:
  musica_dict = {
      "disc_Number": musica["disc_number"],
      "nome": musica["name"],
      "artista": musica["artists"][0]["name"],
      "link": musica["uri"],
      "track_number": musica["track_number"]
    }
  musicas_lista_album.append(musica_dict)

df_album_tracks = spark.createDataFrame(musicas_lista_album)
df_album_tracks.show(truncate=False)

+-------+-----------+------------------------------------+------------------------------------------------------------------+------------+
|artista|disc_Number|link                                |nome                                                              |track_number|
+-------+-----------+------------------------------------+------------------------------------------------------------------+------------+
|Pitbull|1          |spotify:track:6OmhkSOpvYBokMKQxpIGx2|Global Warming (feat. Sensato)                                    |1           |
|Pitbull|1          |spotify:track:2iblMMIgSznA464mNov7A8|Don't Stop the Party (feat. TJR)                                  |2           |
|Pitbull|1          |spotify:track:4yOn1TEcfsKHUJCL2h1r8I|Feel This Moment (feat. Christina Aguilera)                       |3           |
|Pitbull|1          |spotify:track:7fmpKF0rLGPnP7kcQ5ZMm7|Back in Time - featured in "Men In Black 3"                       |4           |
|Pitbull|1          |spotif

### Dataframe com o endpoint "Get Current User's Playlists"

In [16]:
url = "https://api.spotify.com/v1/users/314f34uytt2dd3qhlunzx6cs6kpq/playlists"

params = {
          "limit": 20,
          "offset": 0,
      }

# Fazer requisição
resposta = requests.get(url, params=params, headers=headers)
data = resposta.json()
# Extraia os dados desejados
playlists = []
for item in data["items"]:
  playlists.append({
      "id": item["id"],
      "name": item["name"],
      "description": item["description"],
      "image_url": item["images"][0]["url"],
      "num_tracks": item["tracks"]["total"],
      "public": item["public"],
      "collaborative": item["collaborative"]
    })
df_playlists = spark.createDataFrame(playlists)
df_playlists.show(truncate=False)

+-------------+-----------+----------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----------------+----------+------+
|collaborative|description|id                    |image_url                                                                                                                                                                                  |name             |num_tracks|public|
+-------------+-----------+----------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----------------+----------+------+
|false        |           |7iIz4b05SSO7JGRVCGJwXP|https://mosaic.scdn.co/640/ab67616d0000b2731b7d4dbbf5d96d81026ca278ab67616d0000b273a85ca79ae1931fe1385cc38bab67616d0000b273d9

### Tabela temporária em PySpark a partir do DataFrame de músicas dos artistas

In [17]:
# Cria a tabela temporária de musicas de artistas
df_musicas.createOrReplaceTempView("musicas_artist")

# Consulta a tabela temporária e mostrando
spark.sql("""SELECT * FROM musicas_artist""").show()


+----------------+--------------------+--------------------+--------------------+
|         Artista|                Link|                Nome|               Álbum|
+----------------+--------------------+--------------------+--------------------+
|     Ana Castela|https://open.spot...|Solteiro Forçado ...|Solteiro Forçado ...|
|     Ana Castela|https://open.spot...|Tô Voltando - (Bo...|Tô Voltando (Boia...|
|   Bomba Estéreo|https://open.spot...|     Internacionales|                 Ayo|
|     Ana Castela|https://open.spot...| Fronteira - Ao Vivo|Boiadeira Interna...|
|     Ana Castela|https://open.spot...|Aqui Tem Alguém -...|Boiadeira Interna...|
|El Gran Silencio|https://open.spot...|Super Riddim Inte...|Super Riddim Inte...|
|     Ana Castela|https://open.spot...| Fronteira - Ao Vivo|Fronteira: Boiade...|
|     Darío Gómez|https://open.spot...|         Sobreviviré|El Rey del Despec...|
|     Darío Gómez|https://open.spot...|      Entre Comillas|El Rey del Despec...|
|     Ana Castel

### Tabela temporária em Pyspark a partir do Dataframe de Playlists

In [18]:
# Cria a tabela temporária de musicas de artistas
df_playlists.createOrReplaceTempView("playlists")

# Consulta a tabela temporária e mostrando
spark.sql("""SELECT * FROM playlists""").show()

+-------------+-----------+--------------------+--------------------+-----------------+----------+------+
|collaborative|description|                  id|           image_url|             name|num_tracks|public|
+-------------+-----------+--------------------+--------------------+-----------------+----------+------+
|        false|           |7iIz4b05SSO7JGRVC...|https://mosaic.sc...|Minha playlist #1|         4|  true|
+-------------+-----------+--------------------+--------------------+-----------------+----------+------+



### SQL que retorna os artistas do endpoint "Get Artist" que estão nas Playlists, ordenados por ordem alfabética.

Buscando as musicas da playlist

In [19]:
lista_id_playlist = df_playlists.select('id').collect()
lista_id_playlist = [row["id"] for row in lista_id_playlist]

for id in lista_id_playlist:
  url = f"https://api.spotify.com/v1/playlists/{id}/tracks"
  musicas = []
  # Fazer requisição
  resposta = requests.get(url, headers=headers)
  for item in resposta.json()["items"]:
    musica = {
      "id": item["track"]["id"],
      "musica": item["track"]["name"],
      "artista": item["track"]["artists"][0]["name"],
      "album": item["track"]["album"]["name"]
    }
    musicas.append(musica)
df_playlists_musicas = spark.createDataFrame(musicas)
df_playlists_musicas.show(truncate=False)
df_playlists_musicas.createOrReplaceTempView("playlists_musicas")

+--------------------------------------------------------------+-----------------+----------------------+--------------------------------------------------------------+
|album                                                         |artista          |id                    |musica                                                        |
+--------------------------------------------------------------+-----------------+----------------------+--------------------------------------------------------------+
|A Presença (Ao Vivo)                                          |Gabriela Rocha   |6ELMBm3lZFitAZp03GBBNG|Me Atraiu - Ao Vivo                                           |
|Até Transbordar (Ao Vivo)                                     |Gabriela Rocha   |0CP4vevAzKaN0M5SXEGMXh|Teu Santo Nome - Ao Vivo                                      |
|Acts 2                                                        |Gabriela Rocha   |2SMCSikXj3CYsghcaUVscv|Acts 2                                            

SQL das tabelas

In [20]:
df_consulta = spark.sql("""SELECT a.artista FROM musicas_artist A INNER JOIN playlists_musicas B on A.artista = B.artista ORDER BY artista""")

In [21]:
df_consulta.show(50)

+-----------------+
|          artista|
+-----------------+
|Justin Timberlake|
|Justin Timberlake|
|Justin Timberlake|
|Justin Timberlake|
|Justin Timberlake|
+-----------------+



### Dataframe com o resultado da consulta

In [22]:
df_consulta.write.option("header",True).mode("overwrite").parquet("consulta_sql")